In [1]:
###IN PROGRESS

#Generative Adversarial Network on MNIST

In [2]:
#imports
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [3]:
#inputs and layers
x = tf.placeholder("float", shape=[None, 784]) #input to discriminator
Z = tf.placeholder ("float", shape=[None, 100]) #input to generator


In [8]:
#####################
### DISCRIMINATOR ###
#####################

#784 -> 128 -> 1 
d_w1 = tf.Variable(tf.random_normal(shape=[784, 128]))
d_b1 = tf.Variable(tf.zeros(shape=128))

d_w2 = tf.Variable(tf.random_normal(shape=[128, 1]))
d_b2 = tf.Variable(tf.zeros(shape=1))

#for backprop
theta_d = [d_w1, d_w2, d_b1, d_b2]

#construct discriminator
def discriminator(x):
    '''feedforward discriminator'''
    d_h1 = tf.nn.relu(tf.matmul(x, d_w1) + d_b1)
    d_logistic = tf.matmul(d_h1, d_w2) + d_b2
    d_prob = tf.nn.sigmoid(d_logistic) #probability that input is real
    
    return d_prob, d_logistic

In [9]:
########################
### DC DISCRIMINATOR ###
########################

#784 -> conv -> conv -> 128 -> 1
dc_d_w1 = tf.Variable(tf.random_normal([5, 5, 1, 16]))
dc_d_b1 = tf.Variable(tf.zeros(shape=[16]))

dc_d_w2 = tf.Variable(tf.random_normal([3, 3, 16, 32]))
dc_d_b2 = tf.Variable(tf.zeros(shape=[32]))

dc_d_w3 = tf.Variable(tf.random_normal([7*7*32, 128]))
dc_d_b3 = tf.Variable(tf.zeros(shape=[128]))

dc_d_w4 = tf.Variable(tf.random_normal([128, 1]))
dc_d_b4 = tf.Variable(tf.zeros(shape=[1]))

#for backprop
theta_dc_c = [dc_d_w1, dc_d_b1, dc_d_w2, dc_d_b2, dc_d_w3, dc_d_b3, dc_d_w4, dc_d_b4]

def dc_discriminator(x):
    '''deep convolutional feedforward discriminator'''
    x = tf.reshape(x, shape = [-1, 28, 28, -1])
    conv1 = tf.nn.relu(tf.nn.conv2d(x, dc_d_w1, strides=[1, 2, 2, 1], padding ='SAME' + dc_d_b1))
    conv2 = tf.nn.relu(tf.nn.conv2d(conv1, dc_d_w2, strides=[1, 2, 2, 1], padding ='SAME' + dc_d_b2))
    conv2_out = tf.reshape(conv2, shape=[-1, 7*7*32])
    h3 = tf.nn.relu(tf.matmul(conv2_out, dc_d_w3) + dc_d_b3)
    dc_logistic = tf.matmul(h, dc_d_w4) + dc_d_b4
    dc_prob = tf.nn.sigmoid(dc_logistic)
    
    return dc_prob, dc_logistic

In [6]:
#####################
###   GENERATOR   ###
#####################

#100 -> 128 -> 784 (MNIST image)
g_w1 = tf.Variable(tf.random_normal([100, 128]))
g_b1 = tf.Variable(tf.zeros(shape=[128]))

g_w2 = tf.Variable(tf.random_normal([128, 784]))
g_b2 = tf.Variable(tf.zeros(shape=[784]))


def generator(z):
    g_h1 = tf.nn.relu(tf.matmul(z, g_w1) + g_b1)
    g_logistic = tf.matmul(g_h1, g_w2) + g_b2
    g_prob = tf.nn.sigmoid(g_logistic)
    
    return g_prob
    